In [ ]:
%pip install torch torchvision faiss-cpu pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.0 MB/s eta 0:00:00


 resnet paper link:
 https://www.youtube.com/watch?v=uztrVK1BhGw
 https://dspace.mit.edu/bitstream/handle/1721.1/123067/1127292128-MIT.pdf

#Input Handling

####Classification

In [ ]:
from PIL import Image
import numpy as np
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms

#loadin pretrained resnet50
resnet50 = models.resnet50(pretrained=True)#weights=ResNet50_Weights.IMAGENET1K_V1)
resnet50.eval()

# load image
#img = Image.open('malle_picture_1.jpg')

''' my prepocessing
# resize img
resized_img = img.resize((224,224))


# converts imagge to numpy for computation
np_resized_img = np.array(resized_img)

# normalize image
mean =  mean = [0.485, 0.456, 0.406]  #np.mean(np_resized_img, axis=(0,1))
std = std = [0.229, 0.224, 0.225] #np.std(np_resized_img,axis=(0,1))

# center pixels around zero with a std of 1
normalized_img = ((np_resized_img/255) - mean)/ std'''

# Image preprocessing, transform maintains resnet channel order (C, H, W)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                        std=[0.229, 0.224, 0.225])
])

#load image
img = Image.open('pic1.jpg')
img_tensor = transform(img).unsqueeze(0)  # Add batch dimension so final shape is (1,3,224,224)

# s
with torch.no_grad():
    logits = resnet50(img_tensor)  # Shape: (1, 1000)
    probs = torch.nn.functional.softmax(logits, dim=1)
    top_prob, top_class = torch.max(probs, 1)

print(f"Predicted class index: {top_class.item()}, probability: {top_prob.item():.4f}")

print()


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted class index: 207, probability: 0.7625



####Embedding

In [ ]:
img1 = Image.open('pic1.jpg')
img1_tensor = transform(img1).unsqueeze(0)

img2 = Image.open('pic4.JPG')
img2_tensor = transform(img2).unsqueeze(0)



with torch.no_grad():
  img1_embed = resnet50(img1_tensor).flatten()

  img2_embed = resnet50(img2_tensor).flatten()

  print(img1_embed.shape)

torch.Size([1000])


####Search


In [ ]:
import torch.nn.functional as F

def cosine_similarity(embedding1, embedding2):
    # Return the cosine similarity between two embeddings
    return F.cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0)).item()

similarity_score = cosine_similarity(img1_embed,img2_embed)
print("Cosine similarity:", similarity_score)

Cosine similarity: 0.958196759223938


In [ ]:
import numpy as np
rand = np.random.rand(4, 3).astype('float32')       # random floats in [0, 1)
rand2 = rand
rand2 = rand2.reshape(3,4)
print(rand, end='\n\n')
print(rand2)


print(rand2[0, 1])     # 20 (row 0, col 1)
print(rand2[:, 1])     # column 1 → [20 50 80]
print(rand2[1:, :2])
print(rand2[0][0])
print(rand2.sum())
print(rand2.reshape((12,1)))

[[0.87775177 0.6554416  0.55007774]
 [0.69709367 0.7461471  0.6986938 ]
 [0.4215019  0.23557462 0.4530734 ]
 [0.49819696 0.17187017 0.05184645]]

[[0.87775177 0.6554416  0.55007774 0.69709367]
 [0.7461471  0.6986938  0.4215019  0.23557462]
 [0.4530734  0.49819696 0.17187017 0.05184645]]
0.6554416
[0.6554416  0.6986938  0.49819696]
[[0.7461471  0.6986938 ]
 [0.4530734  0.49819696]]
0.87775177
6.0572696
[[0.87775177]
 [0.6554416 ]
 [0.55007774]
 [0.69709367]
 [0.7461471 ]
 [0.6986938 ]
 [0.4215019 ]
 [0.23557462]
 [0.4530734 ]
 [0.49819696]
 [0.17187017]
 [0.05184645]]
